In [ ]:
import pandas as pd

In [ ]:
from powerNetworkSolver import PowerNetworkSolver
from powerNetwork import networkTransform, getGenerators, getBuses, getLines,getNumBuses, getNumLines
from powerNetwork import getUndGraph, kron_reduction, edge_removing
from disturbancesGnr import normaldisturbances
from violationChecking import globalcheck
import seaborn as sns; sns.set_theme()
#importing the function for multi-edge removal
from powerNetwork import multi_edge_removing


In [ ]:
import matplotlib.pyplot as plt

In [ ]:

import numpy as np
import copy
from numpy.linalg import inv 
import networkx as nx
from numpy.random import multivariate_normal
from disturbancesGnr import normaldisturbances
from scipy.integrate import odeint
from violationChecking import violationcheck,globalcheck,globalchecksubset
import time 
import math 
from functools import reduce




# 1. Test Case 39, complete network 

In [ ]:
from pypower.api import case39

In [ ]:
network39, subnetwork39 = networkTransform(case39())
df_lines39 = getLines(subnetwork39)
df_buses39 = getBuses(subnetwork39)

In [ ]:
n39, ngnr39, unG39 = getUndGraph(df_buses39,df_lines39, network39)
A39, redL39, redA39  = kron_reduction(n39, ngnr39, unG39)
node_list =  np.arange(ngnr39)+1

#the previous adjacent matrix 
# redA39 = np.array([[0,2.484, 2.858, 2.372, 1.031, 0.222,2.13,13.057,3.676,7.834],
#               [2.484,0,10.494,1.781,0.774,0.166,1.599,1.473,0.903,5.885],
#               [2.858,10.494,0,2.522,1.096,0.236,2.264,1.729,1.144,4.772],
#               [2.372,1.781,2.522,0,17.136,0.535,5.14,1.699,1.755,1.367],
#               [1.031,0.774, 1.096,17.136,0 ,0.232, 2.235, 0.739, 0.763, 0.594],
#               [0.222,0.166,0.236,0.535,0.232,0,1.574, 0.159, 0.164, 0.128],
#               [2.13,1.599,2.264, 5.14, 2.235, 1.574, 0, 1.526, 1.576, 1.227],
#               [13.057,1.473,1.729, 1.699,0.739, 0.159, 1.526, 0, 4.283, 4.159],
#               [3.676, 0.903, 1.144, 1.755, 0.763, 0.164, 1.576, 4.283, 0, 1.37],
#               [7.834, 5.885, 4.772, 1.367, 0.594, 0.128, 1.227, 4.159, 1.37 ,0]])

# redA39 = np.ones((ngnr39,ngnr39))
np.fill_diagonal(redA39, 0)
 

In [ ]:
K = 1
M = np.array([0.2228, 0.1607, 0.1899, 0.1517, 0.1379, 0.1846, 0.1401, 0.1289, 0.183, 2.6526])
D = np.array([0.0332,0.076,0.0862,0.0838,0.0674,0.0862,0.0743,0.0716,0.1101,0.1333])
# Ome = np.zeros(n)
pi = math.pi
t =  2
nn = 100
dt = np.linspace(0, t, nn+1)

sigma = 0.01
theta0 = np.zeros(ngnr39)
omega0 = np.zeros(ngnr39)
OMEGA = np.zeros(ngnr39)
model39 = PowerNetworkSolver(theta0,omega0,redA39, ngnr39, D,M,K,OMEGA)

In [ ]:
from numpy.random import seed

In [ ]:
#generate numerical tracks 
seed(100)
disturbances = normaldisturbances(ngnr39,1,sigma)
sol0 = np.pad(disturbances[0], (ngnr39,0), 'constant', constant_values=(0,0))
single_sol = model39.solkuramoto(sol0,dt)
sol_domega = model39.getDotOmega(single_sol[:,:ngnr39],single_sol[:,ngnr39:],nn)

In [ ]:
for i in range(ngnr39):
    theta = single_sol[:,i]
    plt.plot(dt, theta, "-", label="Theta"+ str(i+1))

plt.xlabel("Time")
plt.ylabel("Theta")    
plt.ylim(-0.1, 0.1)
plt.legend();
plt.title('Phase angles')

In [ ]:
for i in range(ngnr39):
    omega = single_sol[:,ngnr39+i]  
    plt.plot(dt, omega, "-", label="Omega"+ str(i+1))


plt.xlabel("Time")

plt.ylabel("Omega")
plt.ylim(-0.5, 0.5)
plt.legend();
plt.title('Natural rotation frequencies')

In [ ]:
for i in range(ngnr39):
    domega = sol_domega[:,i]  
    plt.plot(dt, domega, "-", label="Omega Dot"+ str(i+1))

plt.xlabel("Time")
plt.ylabel("Omega Dot")
plt.ylim(-5, 5)
plt.legend();
plt.title('Change Rates of Natural rotation frequencies')

In [ ]:
# simulation to calculate failure rates 
check_times = 100
KK =3000#repetition times  
thres = np.array([0.2,2])#thres1 is for omega, thres2 is for omega_dot
rates39 = model39.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
print(rates39['totaltime'])

In [ ]:
df39 = pd.DataFrame({'Node': node_list,'RoCoF': rates39['vcheck_omega'] , 'AFV': rates39['vcheck_theta'] ,'AV': rates39['vcheck_any']})
print(df39.to_latex(index=False))

In [ ]:
test=pd.DataFrame()

In [ ]:
# show edge list
# print
edge_list39 = list(unG39.edges)
num_edges39 = getNumLines(getLines(subnetwork39))
rate_list39 = np.zeros((num_edges39,4))
print()
j=0
for i in edge_list39:
    temp_edge = np.array(i)
    temp_G39 = edge_removing(unG39, temp_edge)
    temp_A39, temp_redL39,temp_redA39 = kron_reduction(n39,ngnr39,temp_G39)
    temp_model39 = PowerNetworkSolver(theta0,omega0,temp_redA39,ngnr39,D,M,K,OMEGA)
    temp_rates39 = temp_model39.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)
    temp_df39 = pd.DataFrame({'Node': node_list,'RoCoF': temp_rates39['vcheck_omega'] , 'AFV': temp_rates39['vcheck_theta'] ,'AV': temp_rates39['vcheck_any']})
    rate_list39[j,:]=temp_df39.mean(axis = 0)
    j+=1
    
    

In [ ]:
single_edge_rate39 = pd.DataFrame({'Edge List':edge_list39, 'RoCoF':rate_list39[:,1],'AFV':rate_list39[:,2],'AV':rate_list39[:,3]})

In [ ]:
single_edge_rate39.head()

In [ ]:
single_edge_rate39.to_excel('Single_edge_rate39.xlsx')
# print(single_edge_rate39.to_latex(index=False))

network_test = multi_edge_removing(unG39, [[15,16],[20,34],[23,24]])

# 2. Test Case 39, incomplete networks

## 2.1 Removing (20, 34)

In [ ]:
G3901 = edge_removing(unG39, [20,34])
A3901, redL3901, redA3901  = kron_reduction(n39, ngnr39, G3901)
model3901 = PowerNetworkSolver(theta0,omega0,redA3901, ngnr39,D,M,K,OMEGA)
rates3901 = model3901.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
df3901 = pd.DataFrame({'Node': node_list,'RoCoF': rates3901['vcheck_omega'] , 'AFV': rates3901['vcheck_theta'] ,'AV': rates3901['vcheck_any']})
print(df3901.to_latex(index=False))

## 2.2 Removing (15, 16)

In [ ]:
G3902 = edge_removing(unG39, [15,16])
A3902, redL3902, redA3902  = kron_reduction(n39, ngnr39, G3902)
model3902 = PowerNetworkSolver(theta0,omega0,redA3902, ngnr39, D,M,K,OMEGA)
rates3902 = model3902.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
# disturbances = normaldisturbances(ngnr39,1,sigma)
# sol0 = np.pad(disturbances[0], (ngnr39,0), 'constant', constant_values=(0,0))
single_sol = model3902.solkuramoto(sol0,dt)

In [ ]:
for i in range(ngnr39):
    theta = single_sol[:,i]
    plt.plot(dt, theta, "-", label="omega"+ str(i+1))

plt.xlabel("Time")
plt.ylabel("Omega")    
plt.ylim(-0.1, 0.1)
plt.legend();
plt.title('Natural rotation frequencies')

In [ ]:
for i in range(ngnr39):
    omega = single_sol[:,ngnr39+i]  
    plt.plot(dt, omega, "-", label="omega dot "+ str(i+1))


plt.xlabel("Time")

plt.ylabel("Omega dot")
plt.ylim(-0.4, 0.4)
plt.legend();
plt.title('Derivatives of natural rotation frequencies')

In [ ]:
df3902 = pd.DataFrame({'Node': node_list,'RoCoF': rates3902['vcheck_omega'] , 'AFV': rates3902['vcheck_theta'] ,'AV': rates3902['vcheck_any']})
# print(df3902.to_latex(index=False))

## 2.3 Removing (23, 24)

In [ ]:
G3903 = edge_removing(unG39, [23,24])
A3903, redL3903, redA3903  = kron_reduction(n39, ngnr39, G3903)
model3903 = PowerNetworkSolver(theta0,omega0,redA3903, ngnr39, D,M,K,OMEGA)
rates3903 = model3903.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
df3903 = pd.DataFrame({'Node': node_list,'RoCoF': rates3903['vcheck_omega'] , 'AFV': rates3903['vcheck_theta'] ,'AV': rates3903['vcheck_any']})
# print(df3903.to_latex(index=False))

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
df39.head()

In [ ]:
df39_merged = [df39,df3901,df3902,df3903]
df39merged = reduce(lambda  left,right: pd.merge(left,right,on=['Node'],
                                            how='outer'), df39_merged)
df39merged = df39merged.append(df39merged.mean(axis = 0), ignore_index=True)

# print(df39merged.to_latex(index=False))

In [ ]:
# print(df39merged)
# pd.DataFrame.from_dict(df39_merged)
df39merged.head()

## 2.4 Removing [4, 5],[15, 16]

In [ ]:
G3904 = multi_edge_removing(unG39, ([4,5],[15,16]))
A3904, redL3904, redA3904  = kron_reduction(n39, ngnr39, G3904)
model3904 = PowerNetworkSolver(theta0,omega0,redA3904, ngnr39, D,M,K,OMEGA)
rates3904 = model3904.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
df3904 = pd.DataFrame({'Node': node_list,'RoCoF': rates3904['vcheck_omega'] , 'AFV': rates3904['vcheck_theta'] ,'AV': rates3904['vcheck_any']})
# print(df3901.to_latex(index=False))

In [ ]:
print(df3904.head())

In [ ]:
df3904

## 2.5 Removing [4, 5],[15, 16],[4, 14]

In [ ]:
G3905 = multi_edge_removing(unG39, ([4,5],[15,16],[4,14]))
A3905, redL3905, redA3905  = kron_reduction(n39, ngnr39, G3905)
model3905 = PowerNetworkSolver(theta0,omega0,redA3905, ngnr39, D,M,K,OMEGA)
rates3905 = model3905.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
df3905 = pd.DataFrame({'Node': node_list,'RoCoF': rates3905['vcheck_omega'] , 'AFV': rates3905['vcheck_theta'] ,'AV': rates3905['vcheck_any']})

In [ ]:
df39_merged02 = [df39,df3905]
df39merged02 = reduce(lambda  left,right: pd.merge(left,right,on=['Node'],
                                            how='outer'), df39_merged02)
df39merged02 = df39merged02.append(df39merged02.mean(axis = 0), ignore_index=True)
print(df39merged02.to_latex(index=False))
# df39merged02.to_excel('tuple.xlsx')

## 2.6 Removing [4, 5]

In [ ]:
G3906 = edge_removing(unG39,[4,5])
A3906, redL3906, redA3906  = kron_reduction(n39, ngnr39, G3906)
model3906 = PowerNetworkSolver(theta0,omega0,redA3906, ngnr39, D,M,K,OMEGA)
rates3906 = model3906.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
df3906 = pd.DataFrame({'Node': node_list,'RoCoF': rates3906['vcheck_omega'] , 'AFV': rates3906['vcheck_theta'] ,'AV': rates3906['vcheck_any']})

## 2.7 Removing [4, 14]

In [ ]:
G3907 = edge_removing(unG39,[4,14])
A3907, redL3907, redA3907  = kron_reduction(n39, ngnr39, G3907)
model3907 = PowerNetworkSolver(theta0,omega0,redA3907, ngnr39, D,M,K,OMEGA)
rates3907 = model3907.Simulation(KK, check_times, sigma, thres, t, nn, normaldisturbances)

In [ ]:
df3907 = pd.DataFrame({'Node': node_list,'RoCoF': rates3907['vcheck_omega'] , 'AFV': rates3907['vcheck_theta'] ,'AV': rates3907['vcheck_any']})

In [ ]:
print(np.shape(np.reshape(np.ones(10),(1,10))*np.reshape(np.ones(10),(1,10))))
print(np.shape(np.ones(10)*np.ones(10)))

In [ ]:
df39_merged02 = [df3906,df3907,df3904,df3905]
df39merged02 = reduce(lambda  left,right: pd.merge(left,right,on=['Node'],
                                            how='outer'), df39_merged02)
df39merged02 = df39merged02.append(df39merged02.mean(axis = 0), ignore_index=True)
print(df39merged02.to_latex(index=False))